In [4]:
import pickle
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation

In [5]:
soft_negatives = pickle.load(open('soft_negatives.pkl', 'rb'))
hard_negatives = pickle.load(open('hard_negatives.pkl', 'rb'))
all_data = soft_negatives + hard_negatives

train_samples = [
    InputExample(texts=[triplet[0], triplet[1]], label=1)
    for triplet in all_data
] + [
    InputExample(texts=[triplet[0], triplet[2]], label=1)
    for triplet in all_data
]

In [ ]:
from sentence_transformers import CrossEncoder
from sentence_transformers.cross_encoder.losses import MultipleNegativesRankingLoss, BinaryCrossEntropyLoss

model = CrossEncoder("../models_weights/cross-encoder-russian-msmarco", num_labels=1).to('cuda:1')

# loss = MultipleNegativesRankingLoss(model)
loss = BinaryCrossEntropyLoss(model)

In [ ]:
from sentence_transformers.training_args import BatchSamplers

In [ ]:


from sentence_transformers.cross_encoder import CrossEncoderTrainingArguments

args = CrossEncoderTrainingArguments(
    # Required parameter:
    output_dir="models/reranker-MiniLM-msmarco-v2",
    # Optional training parameters:
    num_train_epochs=10,
    per_device_train_batch_size=64,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    # Optional tracking/debugging parameters:
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    run_name="reranker-MiniLM-msmarco-v2",  # Will be used in W&B if `wandb` is installed
    report_to="none"
)


In [ ]:
from sentence_transformers.cross_encoder import CrossEncoderTrainer
# 6. Create the trainer & start training
trainer = CrossEncoderTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    loss=loss
)
trainer.train()

model.save('./cross_encoder_tuned_v2')


cross_encoder_name = './cross_encoder_tuned'
cross_encoder_model = CrossEncoder(cross_encoder_name, num_labels=1).to('cuda')
